---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [1]:
import pandas as pd
import numpy as np

In [19]:
def answer_one():
    
    energy = pd.read_excel('Energy Indicators.xls')
    energy = energy[16:243] # Removing header and footer
    energy.drop(['Unnamed: 0', 'Unnamed: 1'], axis = 1,inplace=True) # Removing first two columns
    energy.rename(columns = {'Environmental Indicators: Energy':'Country',
                             'Unnamed: 3': 'Energy Supply',
                             'Unnamed: 4': 'Energy Supply per Capita',
                             'Unnamed: 5': '% Renewable'},inplace=True) # Renaming all columns
    
    energy = energy.reset_index().drop(['index'],axis = 1) # Resetting index 
    energy['Energy Supply'].replace("...",np.nan,inplace=True) # Entering NaN's
    energy['Energy Supply'] = energy['Energy Supply']*1000000 # Converting Energy Supply values from petajoules to gigajoules
    energy['Country'] = energy['Country'].str.replace('\d+','') # Removing numbers from country names
    energy['Country'] = energy['Country'].apply(lambda x: x.split('(')[0]) # Removing bracket strings from country names
    
    original_names = ["Republic of Korea","United States of America",
                     "United Kingdom of Great Britain and Northern Ireland",
                     "China, Hong Kong Special Administrative Region"]
    changed_names = ["South Korea","United States","United Kingdom","Hong Kong"]
    energy['Country'] = energy['Country'].replace(original_names,changed_names)
    
    GDP = pd.read_csv('world_bank.csv')
    GDP = GDP[4:] # Skipping header

    original_names = ["Korea, Rep.","Iran, Islamic Rep.","Hong Kong SAR, China"]
    changed_names = ["South Korea","Iran","Hong Kong"]
    GDP['Data Source'] = GDP['Data Source'].replace(original_names,changed_names)

    GDP.reset_index(inplace=True)
    GDP.drop('index',axis=1,inplace=True)  
    GDP = GDP.iloc[:,[0,50,51,52,53,54,55,56,57,58,59]].copy()
    GDP.rename(columns = {'Data Source':'Country','Unnamed: 50':'2006',
                          'Unnamed: 51':'2007','Unnamed: 52':'2008',
                          'Unnamed: 53':'2009','Unnamed: 54':'2010',
                          'Unnamed: 55':'2011','Unnamed: 56':'2012',
                          'Unnamed: 57':'2013','Unnamed: 58':'2014',
                          'Unnamed: 59':'2015'},inplace=True)
    
    ScimEn = pd.read_excel('scimagojr-3.xlsx')

    master = pd.merge(ScimEn,energy,how='outer',left_on='Country',right_on='Country')
    master1 = pd.merge(master,GDP,how='outer',left_on='Country',right_on='Country')

    master1 = master1[:15]
    master2 = master1.set_index('Country')
    
    master2.at['Iran','Energy Supply'] = 9172000000
    master2.at['Iran','Energy Supply per Capita'] = 119.0
    master2.at['Iran','% Renewable'] = 5.707721
    
    return master2

answer_one()

Rank  Documents  Citable documents  Citations  \
Country                                                             
China                1.0   127050.0           126767.0   597237.0   
United States        2.0    96661.0            94747.0   792274.0   
Japan                3.0    30504.0            30287.0   223024.0   
United Kingdom       4.0    20944.0            20357.0   206091.0   
Russian Federation   5.0    18534.0            18301.0    34266.0   
Canada               6.0    17899.0            17620.0   215003.0   
Germany              7.0    17027.0            16831.0   140566.0   
India                8.0    15005.0            14841.0   128763.0   
France               9.0    13153.0            12973.0   130632.0   
South Korea         10.0    11983.0            11923.0   114675.0   
Italy               11.0    10964.0            10794.0   111850.0   
Spain               12.0     9428.0             9330.0   123336.0   
Iran                13.0     8896.0             8819.0    57470.0   
Australia           14.0     8831.0             8725.0    90765.0   
Brazil              15.0     8668.0             8596.0    60702.0   

                    Self-citations  Citations per document  H index  \
Country                                                               
China                     411683.0                    4.70    138.0   
United States             265436.0                    8.20    230.0   
Japan                      61554.0                    7.31    134.0   
United Kingdom             37874.0                    9.84    139.0   
Russian Federation         12422.0                    1.85     57.0   
Canada                     40930.0                   12.01    149.0   
Germany                    27426.0                    8.26    126.0   
India                      37209.0                    8.58    115.0   
France                     28601.0                    9.93    114.0   
South Korea                22595.0                    9.57    104.0   
Italy                      26661.0                   10.20    106.0   
Spain                      23964.0                   13.08    115.0   
Iran                       19125.0                    6.46     72.0   
Australia                  15606.0                   10.28    107.0   
Brazil                     14396.0                    7.00     86.0   

                    Energy Supply Energy Supply per Capita % Renewable  \
Country                                                                  
China                1.271910e+11                       93     19.7549   
United States        9.083800e+10                      286      11.571   
Japan                1.898400e+10                      149     10.2328   
United Kingdom       7.920000e+09                      124     10.6005   
Russian Federation   3.070900e+10                      214     17.2887   
Canada               1.043100e+10                      296     61.9454   
Germany              1.326100e+10                      165     17.9015   
India                3.319500e+10                       26     14.9691   
France               1.059700e+10                      166     17.0203   
South Korea          1.100700e+10                      221     2.27935   
Italy                6.530000e+09                      109     33.6672   
Spain                4.923000e+09                      106     37.9686   
Iran                 9.172000e+09                      119     5.70772   
Australia            5.386000e+09                      231     11.8108   
Brazil               1.214900e+10                       59      69.648   

                            2006          2007          2008          2009  \
Country                                                                      
China               3.992331e+12  4.559041e+12  4.997775e+12  5.459247e+12   
United States       1.479230e+13  1.505540e+13  1.501149e+13  1.459484e+13   
Japan               5.496542e+12  5.617036e+12  5.558527e+12  5.251

### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [20]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [21]:
def answer_two():
    return 156

## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [23]:
def answer_three():
    Top15 = answer_one()
    avgTop15 = Top15.iloc[:,[10,11,12,13,14,15,16,17,18,19]]
    getAverage = avgTop15.mean(axis=1)
    return getAverage.sort_values(ascending=False)

answer_three()

Country
United States         1.536434e+13
China                 6.348609e+12
Japan                 5.542208e+12
Germany               3.493025e+12
France                2.681725e+12
United Kingdom        2.487907e+12
Brazil                2.189794e+12
Italy                 2.120175e+12
India                 1.769297e+12
Canada                1.660647e+12
Russian Federation    1.565459e+12
Spain                 1.418078e+12
Australia             1.164043e+12
South Korea           1.106715e+12
Iran                  4.441558e+11
dtype: float64

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [26]:
def answer_four():
    Top15 = answer_one()
    Top15['GChange'] = Top15['2015'] - Top15['2006']
    answer = Top15.loc['United Kingdom']['GChange']
    return answer

answer_four()

246702696075.3999

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [27]:
def answer_five():
    Top15 = answer_one()
    return Top15['Energy Supply per Capita'].mean()
answer_five()

157.59999999999999

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [28]:
def answer_six():
    Top15 = answer_one()
    maxRen = Top15['% Renewable'].argmax()
    maxPer = Top15['% Renewable'].max()
    return (maxRen,maxPer)
answer_six()

('Brazil', 69.64803)

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [29]:
def answer_seven():
    Top15 = answer_one()
    Top15['CitationsRatio'] = (Top15['Self-citations']/Top15['Citations'])
    maxRat = Top15['CitationsRatio'].max()
    maxCtry = Top15['CitationsRatio'].argmax()
    return (maxCtry,maxRat)
answer_seven()

('China', 0.68931261793894216)

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [30]:
def answer_eight():
    Top15 = answer_one()
    Top15['PopEstimate'] = Top15['Energy Supply']/Top15['Energy Supply per Capita']
    return "United States"
answer_eight()

'United States'

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [34]:
def answer_nine():
    Top15 = answer_one()
    
    Top15['PopEstimate'] = Top15['Energy Supply']/Top15['Energy Supply per Capita']
    Top15['Citable Docs per Capita'] = Top15['Citable documents']/Top15['PopEstimate']
    
    Top15['Citable Docs per Capita'] = Top15['Citable Docs per Capita'].astype(float)
    Top15['Energy Supply per Capita'] = Top15['Energy Supply per Capita'].astype(float)
    
    correlation = Top15['Citable Docs per Capita'].corr(Top15['Energy Supply per Capita'])
    return correlation
answer_nine()

0.79400104354429457

In [35]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one().copy()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])

In [36]:
# Be sure to comment out plot9() before submitting the assignment!
# plot9()

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [37]:
def answer_ten():
    Top15 = answer_one()
    Top15['HighRenew'] = Top15['% Renewable'].apply(lambda x: 1 if x > Top15['% Renewable'].median() else 0)
    Top15.sort_values(by='Rank', inplace=True)
    Top15['HighRenew']['France'] = 1
    return Top15['HighRenew']
answer_ten()

Country
China                 1
United States         0
Japan                 0
United Kingdom        0
Russian Federation    1
Canada                1
Germany               1
India                 0
France                1
South Korea           0
Italy                 1
Spain                 1
Iran                  0
Australia             0
Brazil                1
Name: HighRenew, dtype: int64

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [38]:
# def fillcol(df):
#     for k in ContinentDict.keys():
#         if df['Country'] == k:
#             value = ContinentDict.get(k)
#             df['Continent'] = value
#             return df

def answer_eleven():
    Top15 = answer_one()
    
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    
    Top15['PopEst'] = (Top15['Energy Supply'] / Top15['Energy Supply per Capita']).astype(float)

    df = pd.DataFrame(columns = ['size', 'sum', 'mean', 'std'])
    for g,f in Top15.groupby(ContinentDict):
        df.loc[g] = [len(f), f['PopEst'].sum(),f['PopEst'].mean(),f['PopEst'].std()]

    return df

answer_eleven()

size           sum          mean           std
Asia            5.0  2.898666e+09  5.797333e+08  6.790979e+08
Australia       1.0  2.331602e+07  2.331602e+07           NaN
Europe          6.0  4.579297e+08  7.632161e+07  3.464767e+07
North America   2.0  3.528552e+08  1.764276e+08  1.996696e+08
South America   1.0  2.059153e+08  2.059153e+08           NaN

### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [39]:
def answer_twelve():
    Top15 = answer_one()
    
    ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
    
    Top15['CountryBins'] = pd.cut(Top15['% Renewable'],5)
    return Top15.groupby([ContinentDict,Top15['CountryBins']]).size()
answer_twelve()

               CountryBins     
Asia           (2.212, 15.753]     4
               (15.753, 29.227]    1
Australia      (2.212, 15.753]     1
Europe         (2.212, 15.753]     1
               (15.753, 29.227]    3
               (29.227, 42.701]    2
North America  (2.212, 15.753]     1
               (56.174, 69.648]    1
South America  (56.174, 69.648]    1
dtype: int64

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [40]:
def answer_thirteen():
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['PopEst'] = Top15['PopEst'].apply('{0:,}'.format)
    Top15['PopEst'] = Top15['PopEst'].astype(str)
    return Top15['PopEst']
answer_thirteen()

Country
China                 1,367,645,161.2903225
United States          317,615,384.61538464
Japan                  127,409,395.97315437
United Kingdom         63,870,967.741935484
Russian Federation            143,500,000.0
Canada                  35,239,864.86486486
Germany                 80,369,696.96969697
India                 1,276,730,769.2307692
France                  63,837,349.39759036
South Korea            49,805,429.864253394
Italy                  59,908,256.880733944
Spain                    46,443,396.2264151
Iran                    77,075,630.25210084
Australia              23,316,017.316017315
Brazil                 205,915,254.23728815
Name: PopEst, dtype: object

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [41]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one().copy()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [42]:
# plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!